In [1]:
import os
os.chdir("C:/Users/daess/Desktop/study/공모전/2022 빅콘테스트/Data")

In [2]:
import pandas as pd
import collections

JT_MT_ACCTO_TRRSRT_SCCNT_LIST
- 제주 관광수요예측을 위한 비짓제주 조회 데이터


In [3]:
visit_hits = pd.read_csv('Jeju_Data/JT_MT_ACCTO_TRRSRT_SCCNT_LIST.csv', encoding = 'ANSI')

In [4]:
# 필요한 변수들만 추출
col_list = ['CL_NM', 'AREA_NM', 'ADDR', 'ALL_TOTAL_CO']
visit_hits = visit_hits[col_list]

### **주소, 장소 변수**

엑셀로 살펴보면 주소 형식이 맞지 않는 곳들이 있어 수정하겠습니다.

동문수산시장의 경우, 잘못된 주소(인천 강화군)으로 되어있으므로 실제 주소 입력
- 도로명 주소가 없어, 지번 주소 입력

In [5]:
visit_hits.loc[visit_hits.AREA_NM == '동문수산시장', 'ADDR'] = '제주 제주시 이도1동 1349-60'

In [6]:
visit_hits.loc[visit_hits.AREA_NM == '쉬림프박스', 'ADDR'] = '제주 제주시 구좌읍 평대리 2033-7'

일레인호텔의 경우, 도로명 주소가 영어로 되어있어 한국어로 바꿈

In [7]:
visit_hits.loc[visit_hits.AREA_NM == '일레인호텔', 'ADDR'] = '제주 서귀포시 일주서로 660'

ADDR 변수를 살펴보면 결측치가 24개 존재함
- 주소가 결측치인 장소들을 살펴보면 일부 행에만 주소가 존재하므로, 결측치는 데이터에 있는 주소들로 채우겠습니다

In [8]:
# 도로명 주소가 결측치인 관광지들의 이름 출력
non_names = list(set(visit_hits[visit_hits['ADDR'].isnull()].AREA_NM))

In [9]:
non_names

['무릉자전거도로(환상자전거길 3구간)',
 '렛츠런팜 양귀비꽃',
 '비양도 연평리 야영지',
 '새별헤이요목장',
 '신흥리 동백 & 향나무길',
 '참굴비조형물',
 '서영아리오름 / 용와이오름',
 '월라봉 진지동굴 (안덕면)']

In [10]:
for name in non_names:
    addr = visit_hits.loc[(visit_hits.AREA_NM == name) & (visit_hits['ADDR'].notnull()), 'ADDR']
    addr = addr.iloc[0] # 제일 위에 있는 주소 추출
    visit_hits.loc[(visit_hits.AREA_NM == name) & (visit_hits['ADDR'].isnull()), 'ADDR'] = addr

In [11]:
# 결측치 처리가 잘 되었는지 확인
visit_hits[visit_hits['ADDR'].isnull()]

,CL_NM,AREA_NM,ADDR,ALL_TOTAL_CO


In [12]:
# 추자도의 경우, 하루에 배편이 2개뿐이므로 제외
chuja = visit_hits[visit_hits['ADDR'].str.contains('추자')].index
visit_hits.drop(chuja, inplace = True)

In [13]:
len(visit_hits)

21644

이후에 발생할 인덱스 오류 방지를 위해 인덱스 초기화

In [14]:
visit_hits.reset_index(drop = True, inplace = True)

- 장소 이름과 주소 변수의 경우, 띄어쓰기가 제각각이여서 공백 모두 없앰
- 주소 변수의 경우, 불필요한 정보 제거
  - 제주특별자치도, 제주도, 제주(일부 값은 도명이 아닌 시명으로 시작해서 통일하기 위해)
  - 괄호 이후 값들

In [15]:
needless = ['제주특별자치도', '제주도', '제주']

In [16]:
# 루트에 공백 삭제하기 전 장소명을 반영할 수 있도록 띄어쓰기 전의 장소명을 별도로 저장
before_names = []

# 장소, 주소 처리
for i in range(len(visit_hits)):
    # 장소 처리
    visit_hits.loc[i, 'AREA_NM'] = visit_hits.loc[i, 'AREA_NM'].replace(' ', '')
    
    # 주소 처리
    addr = visit_hits.loc[i, 'ADDR'].lstrip().split('(')
    addr_0 = addr[0].split(' ') # 필요한 주소 리스트
    if addr_0[0] in needless:
        visit_hits.loc[i, 'ADDR'] = ''.join(addr_0[1:])
    else:
        visit_hits.loc[i, 'ADDR'] = ''.join(addr_0)


데이터 확인 후, 오류 발견
- 주소가 제주특별자치도인 곳은 주소가 결측치인 경우
- 주소가 '제주 특별자치도', '제주툭별자치도', 로 시작하는 경우
- 주소의 띄어쓰기 오류
- 사이트 정보 오타로 인한 오류

In [17]:
# 도령모루과 종남마을, 연미마을(오라리방화사건현장)의 경우, 지도 상에 없는 곳이므로 삭제
del_addr1 = visit_hits[visit_hits['AREA_NM'].str.contains('도령모루|종남마을')].index
visit_hits.drop(del_addr1, inplace = True)

In [18]:
del_addr2 = visit_hits['AREA_NM'].isin(['연미마을(오라리방화사건현장)'])
visit_hits = visit_hits[~del_addr2]

In [19]:
# 제주 특별자치도로 시작한 주소 처리
change_addr1 =  visit_hits.loc[visit_hits.AREA_NM == 'ZeZuZip', 'ADDR'].str.replace('특별자치도', '')
visit_hits.loc[visit_hits.AREA_NM == 'ZeZuZip', 'ADDR'] = change_addr1

In [20]:
# 제주툭별자치도로 시작한 주소 처리
change_addr2 =  visit_hits.loc[visit_hits.AREA_NM == '중문스테이', 'ADDR'].str.replace('제주툭별자치도', '')
visit_hits.loc[visit_hits.AREA_NM == '중문스테이', 'ADDR'] = change_addr2

In [21]:
# 오타 처리 : 거쉬포시 -> 서귀포시
error_addr = visit_hits.loc[visit_hits.AREA_NM == '성산오조리해안', 'ADDR'].str.replace('거쉬', '서귀')
visit_hits.loc[visit_hits.AREA_NM == '성산오조리해안', 'ADDR'] = error_addr

In [22]:
# 띄어쓰기 오류1
visit_hits.loc[visit_hits.AREA_NM == '산양곶자왈', 'ADDR'] = '제주시한경면청수리956-6'

In [23]:
# 띄어쓰기 오류2
visit_hits.loc[visit_hits.AREA_NM == '천지연기정길', 'ADDR'] = '서귀포시서귀동961-33'

In [24]:
# 띄어쓰기 오류3
visit_hits.loc[visit_hits.AREA_NM == '하모씨워킹', 'ADDR'] = '서귀포시대정읍최남단해안로130'

In [25]:
# 주소, 장소 목록
ADDR_list = list(set(visit_hits.ADDR))
AREA_list = list(set(visit_hits.AREA_NM))

In [26]:
# 총 주소와 장소 수
print(len(ADDR_list), len(AREA_list))

3604 3973


In [27]:
# 주소는 같으나 장소 이름이 다른 경우가 있음
# 주소에 해당하는 장소이름들이 2개 이상인 경우, 마지막 장소 이름(최근)으로 통일
for addr in ADDR_list:
    name_list = list(set(visit_hits.loc[visit_hits.ADDR == addr, 'AREA_NM']))
    if len(name_list) > 1:
        visit_hits.loc[visit_hits.ADDR == addr, 'AREA_NM'] = name_list[-1]
    

In [28]:
visit_hits.loc[visit_hits.ADDR == '제주시한경면낙수로97', 'AREA_NM']

3040     낙천아홉굿마을의자공원
3377     낙천아홉굿마을의자공원
6560     낙천아홉굿마을의자공원
6897     낙천아홉굿마을의자공원
7171     낙천아홉굿마을의자공원
7669     낙천아홉굿마을의자공원
10609    낙천아홉굿마을의자공원
10610    낙천아홉굿마을의자공원
14302    낙천아홉굿마을의자공원
14303    낙천아홉굿마을의자공원
18035    낙천아홉굿마을의자공원
18036    낙천아홉굿마을의자공원
Name: AREA_NM, dtype: object

In [29]:
# 장소 이름은 같으나 주소가 다른 경우
# 장소에 해당하는 주소가 2개 이상인 경우, 마지막 주소(최근)로 통일
for area in AREA_list:
    addr_list = list(set(visit_hits.loc[visit_hits.AREA_NM == area, 'ADDR']))
    if len(addr_list) > 1:
        visit_hits.loc[visit_hits.AREA_NM == area, 'ADDR'] = addr_list[-1]

In [30]:
visit_hits.loc[visit_hits.ADDR == '제주시한경면낙수로97', 'AREA_NM']

3040     낙천아홉굿마을의자공원
3377     낙천아홉굿마을의자공원
6560     낙천아홉굿마을의자공원
6897     낙천아홉굿마을의자공원
7171     낙천아홉굿마을의자공원
7669     낙천아홉굿마을의자공원
10609    낙천아홉굿마을의자공원
10610    낙천아홉굿마을의자공원
14302    낙천아홉굿마을의자공원
14303    낙천아홉굿마을의자공원
18035    낙천아홉굿마을의자공원
18036    낙천아홉굿마을의자공원
Name: AREA_NM, dtype: object

In [31]:
# 총 장소와 주소수를 확인하면 같아졌음을 알 수 있음
ADDR_list = list(set(visit_hits.ADDR))
AREA_list = list(set(visit_hits.AREA_NM))

print(len(ADDR_list), len(AREA_list))

3561 3561


In [32]:
# 최종 정해진 장소명과 원본 장소명 합치기
# before_names 생성 이후 삭제한 장소명 반영하기
print(len(visit_hits), len(before_names))

21626 21644


### **분류코드 변수**

In [42]:
visit_hits.groupby(['CL_NM', 'AREA_NM']).sum()

ALL_TOTAL_CO
CL_NM AREA_NM                    
관광지   (주)제주해양레저투명카약          2033
      1100고지(휴게소)            2679
      1100고지습지               3860
      4·3해원방사탑                280
      5.16도로숲터널              2454
...                           ...
음식점   흑돼지촌                    143
      흑돼지킹생구이전문점               99
      흑소랑                     240
      흑염소마루                   136
      흑한우명품관                  418

[3681 rows x 1 columns]

장소별 분류명이 여러개일 경우, 각 분류명의 개수를 계산하여 개수가 최대인 분류명 선택
- 만약 개수가 같다면 가장 먼저 나온 분류명으로

In [43]:
for area in AREA_list:
    area_cl = list(visit_hits.loc[visit_hits.AREA_NM == area, 'CL_NM'])
    cl_counts = collections.Counter(area_cl)
    visit_hits.loc[visit_hits.AREA_NM == area, 'CL_NM'] = cl_counts.most_common()[0][0]

In [44]:
visit_hits.groupby(['CL_NM', 'AREA_NM']).sum()

ALL_TOTAL_CO
CL_NM AREA_NM                    
관광지   (주)제주해양레저투명카약          2033
      1100고지(휴게소)            2679
      1100고지습지               3860
      4·3해원방사탑                280
      5.16도로숲터널              2454
...                           ...
음식점   흑돼지촌                    143
      흑돼지킹생구이전문점               99
      흑소랑                     240
      흑염소마루                   136
      흑한우명품관                  418

[3561 rows x 1 columns]

### **장소별 2년간 총 조회수 계산**
데이터가 준비되었으니 각 장소별로 2년간 총 조회수 계산하겠습니다
- 21년 9 ~ 11월, 22년 3월 ~ 5월

In [45]:
visit_series = visit_hits.groupby(['CL_NM', 'AREA_NM', 'ADDR']).sum()

In [46]:
visit_df = pd.DataFrame(visit_series)

In [47]:
visit_df.reset_index(level = (0, 1, 2), inplace = True)

주소 중 행정시가 명시되지 않은 값이 있어 행정시 처리해주기

In [48]:
visit_df[~visit_df['ADDR'].str.contains('제주시|서귀포시', na = False)]

,CL_NM,AREA_NM,ADDR,ALL_TOTAL_CO
539,관광지,아홉굿마을낙천리농어촌체험마을,한경면낙수로97,577
738,관광지,제주오름승마랜드,조천읍번영로1734-15,410
2533,음식점,바람이풀에그림그린날,안덕면감산서로59,134


In [49]:
visit_df.loc[visit_df.ADDR == '한경면낙수로97', 'ADDR'] = '제주시한경면낙수로97'
visit_df.loc[visit_df.ADDR == '조천읍번영로1734-15', 'ADDR'] = '제주시조천읍번영로1734-15'
visit_df.loc[visit_df.ADDR == '안덕면감산서로59', 'ADDR'] = '서귀포시안덕면감산서로59'

In [50]:
# 확인
visit_df[~visit_df['ADDR'].str.contains('제주시|서귀포시', na = False)]

,CL_NM,AREA_NM,ADDR,ALL_TOTAL_CO


In [53]:
visit_df[visit_df.duplicated('ADDR')]

,CL_NM,AREA_NM,ADDR,ALL_TOTAL_CO
539,관광지,아홉굿마을낙천리농어촌체험마을,제주시한경면낙수로97,577


In [51]:
# visit_df.to_csv('비짓제주 전처리.csv', encoding = 'utf-8-sig', index = False)